<a href="https://colab.research.google.com/github/Yozora13/Text-To-Image-Genrator-IBM/blob/main/Text_to_image_genrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers transformers accelerate gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.8 MB/s eta 0:00:00


In [ ]:
import torch
import gc
from diffusers import StableDiffusionXLPipeline, StableDiffusionPipeline, DDIMScheduler
import gradio as gr

# Descriptive model names for dropdown
models = {
    "SDXL 1.0 - High-quality, detailed generation (Slow)": "stabilityai/stable-diffusion-xl-base-1.0",
    "SD 1.5 - Fast and reliable, good quality": "runwayml/stable-diffusion-v1-5",
    "SDXL Turbo - Lightning fast, low quality": "stabilityai/sdxl-turbo"
}

loaded_pipelines = {}
current_model = None

# Load selected model and show progress
def load_model(model_name, progress=gr.Progress()):
    global loaded_pipelines, current_model

    if model_name in loaded_pipelines:
        progress(1.0, desc=f"✅ {model_name} already loaded.")
        return f"✅ {model_name} already loaded."

    if current_model and current_model in loaded_pipelines:
        del loaded_pipelines[current_model]
        torch.cuda.empty_cache()
        gc.collect()

    progress(0.1, desc=f"🔄 Downloading {model_name}...")
    model_id = models[model_name]

    if "xl" in model_id.lower():
        pipe = StableDiffusionXLPipeline.from_pretrained(
            model_id, torch_dtype=torch.float16, variant="fp16"
        ).to("cuda")
    else:
        pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            safety_checker=None,
            requires_safety_checker=False
        ).to("cuda")
        pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)

    progress(0.8, desc="⚙️ Finalizing setup...")
    pipe.enable_attention_slicing()

    loaded_pipelines[model_name] = pipe
    current_model = model_name

    progress(1.0, desc=f"✅ {model_name} loaded successfully.")
    return f"✅ {model_name} is ready to use."

# Access current pipeline
def get_pipeline(model_name):
    return loaded_pipelines[model_name]

# 🔄 Preload default model (SDXL 1.0)
print("🔄 Preloading default model...")
load_model("SDXL 1.0 - High-quality, detailed generation (Slow)")
print("✅ SDXL 1.0 is ready.")


In [ ]:
def on_model_change_with_loading(model_name, progress=gr.Progress()):
    status = load_model(model_name, progress)
    return status

In [ ]:
from PIL import Image
import datetime
import random
import gradio as gr
import torch

history_images = []

def generate_image_with_save_and_history(prompt, model_name, guidance_scale=7.5, steps=30, seed=None):
    try:
        seed = int(seed)
    except:
        seed = random.randint(0, 999999)

    pipe = get_pipeline(model_name)
    generator = torch.Generator("cuda").manual_seed(seed)

    image = pipe(
        prompt,
        guidance_scale=guidance_scale,
        num_inference_steps=steps,
        generator=generator
    ).images[0]

    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"generated_{timestamp}.png"
    image.save(filename)

    history_images.insert(0, image)
    if len(history_images) > 20:
        history_images.pop()

    return image, f"🎲 Seed: {seed}", filename, history_images

def clear_history():
    history_images.clear()
    return [], gr.update(value=None, visible=False)

def on_model_change_with_loading(model_name, progress=gr.Progress()):
    return load_model(model_name, progress)


In [ ]:
import random

prompt_suggestions = [
    "futuristic cyberpunk city at night",
    "anime girl under cherry blossom trees, glowing lights",
    "photorealistic dragon flying over mountains",
    "a cozy cabin in snowy forest, ultra-realistic lighting",
    "studio portrait of a samurai in rain, cinematic style",
    "floating islands with waterfalls, fantasy world",
    "pixel art of a wizard casting magic in a forest",
    "mecha robot in a battle scene, anime style",
    "epic space battle with starships, cinematic lighting",
    "surreal dreamscape with melting clocks and clouds"
]

initial_placeholder = random.choice(prompt_suggestions)

with gr.Blocks(css="""
:root {
  --accent-color: #00aaff !important;
  --primary-500: #00aaff !important;
}

body {
    background: linear-gradient(135deg, #0f2027, #203a43, #2c5364);
    font-family: 'Segoe UI', sans-serif;
}
h1 {
    text-align: center;
    font-size: 2.7em;
    margin-top: 20px;
    background: linear-gradient(90deg, #00c6ff, #0072ff);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    text-shadow: 0 0 10px #00c6ff, 0 0 20px #0072ff;
}
.gr-button {
    background: linear-gradient(90deg, #0072ff, #00c6ff);
    color: white;
    border: none;
    font-weight: bold;
    box-shadow: 0 0 20px #00c6ff;
    transition: all 0.3s ease;
    font-size: 1.1rem;
    padding: 0.7rem 2rem;
}
.gr-button:hover {
    box-shadow: 0 0 40px #00c6ff;
    transform: scale(1.05);
    cursor: pointer;
}
.download-btn {
    background: linear-gradient(90deg, #0055ff, #00aaff);
    color: white;
    font-weight: 700;
    font-size: 1.2rem;
    padding: 0.8rem 2.5rem;
    border-radius: 12px;
    box-shadow: 0 0 15px #00aaff;
    border: none;
    margin-top: 15px;
    transition: all 0.3s ease;
}
.download-btn:hover {
    box-shadow: 0 0 30px #00cfff;
    transform: scale(1.1);
    cursor: pointer;
}
.glass-card {
    background: rgba(255, 255, 255, 0.05);
    border-radius: 20px;
    padding: 20px;
    backdrop-filter: blur(12px);
    box-shadow: 0 0 20px rgba(255, 255, 255, 0.1);
    border: 1px solid rgba(255, 255, 255, 0.2);
    margin-top: 20px;
}
input[type="text"]:not([disabled]) {
    color: white !important;
}
#stars {
  position: fixed;
  width: 100%;
  height: 100%;
  z-index: 999;
  pointer-events: none;
}

/* Blue slider styling */
input[type="range"]::-webkit-slider-runnable-track {
    background: linear-gradient(90deg, #0072ff, #00c6ff);
    height: 6px;
    border-radius: 5px;
}
input[type="range"]::-moz-range-track {
    background: linear-gradient(90deg, #0072ff, #00c6ff);
    height: 6px;
    border-radius: 5px;
}
input[type="range"]::-webkit-slider-thumb {
    background-color: white;
    border: 2px solid #00c6ff;
    border-radius: 50%;
    height: 18px;
    width: 18px;
    margin-top: -6px;
    box-shadow: 0 0 10px #00c6ff;
    cursor: pointer;
}
input[type="range"]::-moz-range-thumb {
    background-color: white;
    border: 2px solid #00c6ff;
    border-radius: 50%;
    height: 18px;
    width: 18px;
    box-shadow: 0 0 10px #00c6ff;
    cursor: pointer;
}
""") as demo:

    gr.Markdown("<h1>✨ AI Text-to-Image Generator</h1>")

    gr.HTML("""
    <canvas id="stars"></canvas>
    <script>
    const canvas = document.getElementById("stars");
    const ctx = canvas.getContext("2d");
    canvas.width = window.innerWidth;
    canvas.height = window.innerHeight;
    let stars = [];

    function createStar() {
        return {
            x: Math.random() * canvas.width,
            y: Math.random() * canvas.height,
            radius: Math.random() * 2 + 1,
            alpha: 1,
            dx: (Math.random() - 0.5) * 4,
            dy: (Math.random() - 0.5) * 4
        };
    }

    function burstStars() {
        stars = [];
        for (let i = 0; i < 100; i++) stars.push(createStar());
        requestAnimationFrame(animateStars);
    }

    function animateStars() {
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        stars.forEach((star, i) => {
            ctx.beginPath();
            ctx.arc(star.x, star.y, star.radius, 0, Math.PI * 2);
            ctx.fillStyle = "rgba(255,255,255," + star.alpha + ")";
            ctx.fill();
            star.x += star.dx;
            star.y += star.dy;
            star.alpha -= 0.015;
        });
        stars = stars.filter(s => s.alpha > 0);
        if (stars.length > 0) requestAnimationFrame(animateStars);
    }

    window.burstStars = burstStars;
    </script>
    """)

    with gr.Row():
        prompt = gr.Textbox(
            label="Prompt",
            placeholder=initial_placeholder,
            lines=2,
            scale=2
        )

    with gr.Row():
        model_selector = gr.Dropdown(
            choices=list(models.keys()),
            value="SDXL 1.0 - High-quality, detailed generation (Slow)",
            label="🧠 Choose Model"
        )
        current_model = gr.Textbox(value="✅ SDXL 1.0 loaded", label="Model Status", interactive=False)

    model_selector.change(
        fn=on_model_change_with_loading,
        inputs=model_selector,
        outputs=current_model
    )

    with gr.Accordion("⚙️ Advanced Settings", open=False):
        with gr.Row():
            guidance = gr.Slider(1.0, 15.0, value=7.5, step=0.5, label="🎛️ Guidance Scale")
            steps = gr.Slider(5, 50, value=30, step=1, label="⏱️ Inference Steps")
            seed = gr.Textbox(label="🎲 Seed (optional)", placeholder="Leave blank for random")

    run_button = gr.Button("🚀 Generate Image", size="lg", elem_classes="gr-button")

    with gr.Column(elem_classes="glass-card"):
        image_output = gr.Image(label="🖼️ Output Image", type="pil")
        seed_output = gr.Textbox(label="📌 Generation Info", interactive=False)
        download_button = gr.File(label="📥 Download Image", visible=False, elem_classes="download-btn")
        clear_history_btn = gr.Button("🗑️ Clear History", elem_classes="gr-button")

    gallery = gr.Gallery(label="🖼️ Generation History", show_label=True, columns=4, rows=1, height="auto")

    def show_download(filename):
        return gr.update(value=filename, visible=True)

    def trigger_stars_js():
        return gr.HTML("<script>burstStars()</script>")

    run_button.click(
        fn=generate_image_with_save_and_history,
        inputs=[prompt, model_selector, guidance, steps, seed],
        outputs=[image_output, seed_output, download_button, gallery]
    ).then(
        fn=show_download,
        inputs=[download_button],
        outputs=download_button
    ).then(
        fn=trigger_stars_js,
        outputs=[]
    )

    clear_history_btn.click(
        fn=clear_history,
        inputs=[],
        outputs=[gallery, download_button]
    )

demo.launch()
